In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import scipy.stats as stats
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import warnings
warnings.filterwarnings('ignore') 

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Read datas
median_house_hold_in_come = pd.read_csv('../input/fatal-police-shootings-in-the-us//MedianHouseholdIncome2015.csv', encoding="windows-1252")
percentage_people_below_poverty_level = pd.read_csv('../input/fatal-police-shootings-in-the-us//PercentagePeopleBelowPovertyLevel.csv', encoding="windows-1252")
percent_over_25_completed_highSchool = pd.read_csv('../input/fatal-police-shootings-in-the-us/PercentOver25CompletedHighSchool.csv', encoding="windows-1252")
share_race_city = pd.read_csv('../input/fatal-police-shootings-in-the-us//ShareRaceByCity.csv', encoding="windows-1252")
kill = pd.read_csv('../input/fatal-police-shootings-in-the-us//PoliceKillingsUS.csv', encoding="windows-1252")

## Fakirlik Oranı

In [ ]:
percentage_people_below_poverty_level.head()

In [ ]:
percentage_people_below_poverty_level.info()

In [ ]:
percentage_people_below_poverty_level.poverty_rate.value_counts()

* Fakirlik oranında yer alan "-" değeri anlamsız olduğundan bu özellik değerine sahip gözlem birimlerindeki değerlere 0(sıfır) değeri vererek kurtulmak mümkündür.

In [ ]:
percentage_people_below_poverty_level.poverty_rate.replace(["-"] ,0.0 , inplace = True)

* Veri seti bilgilerinde fakirlik oranını belirten sayısal özelliğin object gözükmesi anlamsızdır. Düzeltmek gerekir.

In [ ]:
percentage_people_below_poverty_level.poverty_rate= percentage_people_below_poverty_level.poverty_rate.astype(float)

* Her bir eyaletin fakirlik oranını bulmak için tekil olarak eyaletler bulunmalıdır.

In [ ]:
area_list = list(percentage_people_below_poverty_level["Geographic Area"].unique())

In [ ]:
len(area_list)

* Eyalet sayısı 51.

* 51 eyaletin kısaltmaları

In [ ]:
print(area_list)

* Eyaletlere ait fakirlik oranını bulunması

In [ ]:
area_poverty_ratio = []
for i in area_list:
    x = percentage_people_below_poverty_level[percentage_people_below_poverty_level["Geographic Area"]==i]
    area_poverty_rate = sum(x.poverty_rate)/len(x)
    area_poverty_ratio.append(area_poverty_rate)

* Görselleştirme için gerekli bu iki özellik üzerinden dataframe oluşturulması yapılabilir.

In [ ]:
data = pd.DataFrame({"area_list" : area_list , "area_poverty_ratio" : area_poverty_ratio})
new_index = (data["area_poverty_ratio"].sort_values(ascending=False)).index.values
sorted_data_1 = data.reindex(new_index)

* Bu özellikleri kullanarak eyaletlerin fakirlik oranlarını görselleştirmek için barplot kullanılabilir.

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data_1["area_list"] , y=sorted_data_1["area_poverty_ratio"])
plt.xticks(rotation = 45)
plt.show()

## Öldürülen İnsanların İsim veya Soyisimlerinden En Sık Olan 15 Tanesi

* İsteneni bize sunacak veri setine genel bir bakış

In [ ]:
kill.head()

* İsimlerin frekansları

In [ ]:
kill.name.value_counts()

* Burada yer alan TK TK ve TK Tk girilen özellik değerleri gerçek isimleri yansıtmamaktadır. Görselleştirme yaparken dahil etmemek gerekir.

In [ ]:
seperate = kill.name[(kill.name != "TK TK")].str.split()
a,b = zip(*seperate)
name_list = a+b
name_count = Counter(name_list)

most_common_names = name_count.most_common(15)
x,y = zip(*most_common_names)
x,y = list(x) , list(y)

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.barplot(x=x, y=y , palette = sns.cubehelix_palette(len(x)));

### Eyaletlere Göre Yetişkin Bireylerin (25+) Lise Mezunu Olma Oranı

In [ ]:
percent_over_25_completed_highSchool.head()

* Uygun formatta olmayan veya eksik veri kontrolü

In [ ]:
percent_over_25_completed_highSchool.percent_completed_hs.value_counts()

* "-" değerine sahip gözlemler var. İstenilen değer değil 0 değeri ile değiştirilebilir.

In [ ]:
percent_over_25_completed_highSchool.percent_completed_hs.replace(["-"],0.0, inplace=True)

In [ ]:
 percent_over_25_completed_highSchool.info()

* Sayısal bir özelliğin değerleri object verildiğinden bunu düzeltmek gerekir.

In [ ]:
percent_over_25_completed_highSchool.percent_completed_hs = percent_over_25_completed_highSchool.percent_completed_hs.astype(float)

* Eyaletlerin isimlerini görselleştirmede kullanmak için alınmalıdır.

In [ ]:
area_list = list(percent_over_25_completed_highSchool["Geographic Area"].unique())

* Tekil olacak şekilde oluşturulan eyalet listesinden faydalanarak 25 yaş üstünde olan lise mezun oranını bulunabilir.

In [ ]:
area_highschool = []
for i in area_list :
    x = percent_over_25_completed_highSchool[percent_over_25_completed_highSchool["Geographic Area"] == i]
    area_highschool_rate = sum(x.percent_completed_hs)/len(x)
    area_highschool.append(area_highschool_rate)

* Sıralama yapmak grafikte karmaşıklığı önleyebilir.

In [ ]:
data = pd.DataFrame({"area_list" : area_list , "area_highschool_ratio" : area_highschool})
new_index = (data["area_highschool_ratio"].sort_values(ascending=True)).index.values
sorted_data_2 = data.reindex(new_index)

* İstenen veri analizini barplot ile gösterilebilir.

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data_2["area_list"] , y=sorted_data_2["area_highschool_ratio"])
plt.xticks(rotation = 90)
plt.xlabel = "Eyaletler"
plt.ylabel = "Lise Mezuniyet Oranı"
plt.title = "Eyaletlerdeki Yetişkinlerdeki Lise Mezuniyet Oranı"
plt.show()

### Eyaletlerdeki Irkların Oranı

In [ ]:
share_race_city.head()

In [ ]:
share_race_city.info()

In [ ]:
share_race_city.replace(["-"] , 0.0 , inplace=True)
share_race_city.replace(["(X)"], 0.0, inplace=True)
share_race_city.loc[:,["share_white","share_black","share_native_american","share_asian","share_hispanic"]] = share_race_city.loc[:,["share_white","share_black","share_native_american","share_asian","share_hispanic"]].astype(float)

In [ ]:
area_list = list(share_race_city["Geographic area"].unique())
share_white = []
share_black = []
share_native_american =[]
share_asian = []
share_hispanic = []

for i in area_list:
    x = share_race_city[share_race_city["Geographic area"] == i]
    share_white.append(sum(x.share_white)/len(x))
    share_black.append(sum(x.share_black)/len(x))
    share_native_american.append(sum(x.share_native_american)/len(x))
    share_asian.append(sum(x.share_asian)/len(x))
    share_hispanic.append(sum(x.share_hispanic)/len(x))

In [ ]:
f,ax = plt.subplots(figsize=(9,15))
sns.barplot(x=share_white, y=area_list , color="yellow" , alpha=0.5 , label="White")
sns.barplot(x=share_black, y=area_list , color="blue" , alpha=0.7 , label="Black")
sns.barplot(x=share_native_american, y=area_list , color="cyan" , alpha=0.6, label="Native American")
sns.barplot(x=share_asian, y=area_list , color="purple" , alpha=0.6 , label="Asian")
sns.barplot(x=share_hispanic, y=area_list , color="red" , alpha=0.6, label="Hispanic")

ax.legend(loc="lower right" , frameon=True)
ax.set(xlabel = "Irk Oranları" , ylabel = "Eyaletler" , title = "Eyaletlerin Irk Dağılımları");

### Her Eyalet İçin Fakirlik Oranı ve Lise Mezuniyet Oranını Karşılaştırılması


In [ ]:
sorted_data_1.head()

In [ ]:
sorted_data_2.head()

In [ ]:
sorted_data_1["area_poverty_ratio"] = sorted_data_1["area_poverty_ratio"]/max(sorted_data_1["area_poverty_ratio"])
sorted_data_2["area_highschool_ratio"] = sorted_data_2["area_highschool_ratio"]/max(sorted_data_2["area_highschool_ratio"])
data = pd.concat([sorted_data_1,sorted_data_2["area_highschool_ratio"]] , axis=1)
data.sort_values("area_poverty_ratio", inplace=True)

In [ ]:
f,ax1 = plt.subplots(figsize=(20,10))
sns.pointplot(x = "area_list" , y = "area_poverty_ratio" , data=data , color="lime" , alpha=0.8)
sns.pointplot(x = "area_list" , y = "area_highschool_ratio" , data=data , color = "red", alpha=0.8)
plt.text(40,0.6, "Lise Mezuniyet Oranı" , color="red" , fontsize=17 , style = "italic")
plt.text(40,0.55, "Fakirlik Oranı" , color="lime" , fontsize=18, style = "italic")
plt.grid()

In [ ]:
g = sns.jointplot(data.area_poverty_ratio , data.area_highschool_ratio , kind = "kde" , size=7).annotate(stats.pearsonr)
plt.show()

In [ ]:
g = sns.jointplot(data.area_poverty_ratio , data.area_highschool_ratio , ratio=3 , size=7, color = "r").annotate(stats.pearsonr)
plt.show()

### Ölen Kişilerin Irklarının Oranı

In [ ]:
kill.head()

In [ ]:
kill.dropna(inplace=True)
labels =kill.race.value_counts().index
colors = ["orange","cyan","red","yellow","green","brown"]
explode = [0,0,0,0,0,0]
sizes = kill.race.value_counts().values

In [ ]:
plt.figure(figsize=(7,7))
plt.pie(sizes,explode=explode,labels=labels,colors=colors,autopct="%1.1f%%");

### Fakirlik Oranı İle Liseden Mezun Olma Oranı

In [ ]:
sns.lmplot(x="area_poverty_ratio" , y = "area_highschool_ratio" , data=data)
plt.show()

In [ ]:
sns.kdeplot(data.area_poverty_ratio , data.area_highschool_ratio, shade=True , cut=5)
plt.show()

In [ ]:
pal = sns.cubehelix_palette(2,rot=-.5, dark=-.3)
sns.violinplot(data=data, palette=pal, inner="points")
plt.show()

In [ ]:
data.head()

### Korelasyon Haritası

In [ ]:
sns.pairplot(data)
plt.show()

In [ ]:
f,ax = plt.subplots(figsize=(5,5))
sns.heatmap(data.corr() , annot=True , linewidth=.5 ,linecolor = "red", fmt=".1f" , ax=ax)
plt.show()

#### Ölüm Şekilleri ve Cinsiyetlerine Göre Ölen Kişilerin Yaş Dağılımları

In [ ]:
sns.boxplot(x="gender" , y="age" , hue="manner_of_death" , data=kill , palette = "PRGn")
plt.show()

In [ ]:
sns.swarmplot(x="gender" , y="age" , hue="manner_of_death" , data=kill)
plt.show()

### Ölüme Sebep Olan Silahların Kullanım Frekansları

In [ ]:
kill.manner_of_death.value_counts()

In [ ]:
sns.countplot(kill.manner_of_death)
plt.show()

### Öldürülen Kişilerin Kullandığı Silahlar

In [ ]:
armed = kill.armed.value_counts()
plt.figure(figsize=(10,7))
sns.barplot(x=armed[:7].index, y=armed[:7].values)
plt.show()

### En Tehlikeli Şehirler

In [ ]:
city = kill.city.value_counts()
plt.figure(figsize=(10,7))
sns.barplot(x=city[:12].index , y=city[:12].values)
plt.xticks(rotation=45)
plt.show()